In [2]:
# import os
import pickle
# actors = os.listdir('data')
# filenames = []
# for actor in actors:
#     for file in os.listdir(os.path.join('data', actor)):
#         filenames.append(os.path.join('data', actor, file))

# pickle.dump(filenames, open("filename.pkl", "wb"))
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50

In [3]:
filenames = pickle.load(open("filename.pkl", "rb"))

In [4]:
filenames[0]

'data\\Aamir_Khan\\Aamir.100.jpg'

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [6]:
model = resnet50(pretrained=True)
# Remove the classification layer and add global average pooling
model = nn.Sequential(*list(model.children())[:-2], nn.AdaptiveAvgPool2d((1, 1)))
model = model.to(device)
model.eval()

# Define the image transformations
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


c:\Users\swara\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\swara\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [8]:
from PIL import Image

In [9]:
def img_to_vector(img, model):
    img = Image.fromarray(img)
    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        features = model(img_tensor)
    return features.cpu().numpy().flatten()

In [16]:
import cv2
features = []
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
for file in filenames:
    sample_img = cv2.imread(file)
    gray_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 1:
        x, y, width, height = tuple(faces[0])
        face = sample_img[y:y+height,x:x+width]
    else:
        face = sample_img
    result = img_to_vector(face, model)
    features.append(result)
pickle.dump(features, open('embedding.pkl','wb'))        

1
1
1
0


IndexError: tuple index out of range